d ##### Usecase:Get sales data by customers
###### Solutions are given for following problem statements
###### Problem statement-1:Get sales data for customers per day
###### Problem statement-2:Get sales data for customers per month
###### Problem statement-3:Get sales data for customers per year
###### Problem statement-4:Get monthly sales data from most visited customers
###### Problem statement-5:Get monthly sales data from Top10 revenue generating customers 
###### Problem statement-6:Get daily sales data from fourth quarter of 2013 and second quarter of 2014 where customer_state is CA
###### Problem statement-7:Get monthly sales data from Top10 revenue generating cities
###### Problem statement-8:Get Top10 revenue generating zipcodes and its corresponding cities and states
###### Problem statement-9:Get revenue of  each zipcodes of state COLORADO and its corresponding cities 
###### Problem statement-10:Get total revenue of each state in last quarter of 2013
###### Problem statement-11:Get the list of inactive customers

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import sum,round,avg,rank, col
from pyspark.sql.window import Window
from pyspark.sql.functions import *
from pyspark.sql.functions import unix_timestamp, lit
spark = SparkSession.builder.master('local').appName('SalesDatabycustomers').enableHiveSupport().getOrCreate()

In [3]:
ordersdf = spark.read.csv("/FileStore/tables/retaildbtext/Orders.txt", sep=',',schema ='order_id int,order_date string,order_customer_id int, order_status string')
ordersdf.show(5,False)

+--------+----------+-----------------+---------------+
order_id|order_date|order_customer_id|order_status |
+--------+----------+-----------------+---------------+
1 |2013-07-25|11599 |CLOSED |
2 |2013-07-25|256 |PENDING_PAYMENT|
3 |2013-07-25|12111 |COMPLETE |
4 |2013-07-25|8827 |CLOSED |
5 |2013-07-25|11318 |COMPLETE |
+--------+----------+-----------------+---------------+
only showing top 5 rows

In [4]:
orderItemsdf = spark.read.csv("/FileStore/tables/retaildbtext/Order_Items.txt", sep=',', schema='order_item_id int, order_item_order_id int, order_item_product_id  int, order_item_quantity int,order_item_subtotal float,order_item_product_price float')
orderItemsdf.show(5,False)

+-------------+-------------------+---------------------+-------------------+-------------------+------------------------+
order_item_id|order_item_order_id|order_item_product_id|order_item_quantity|order_item_subtotal|order_item_product_price|
+-------------+-------------------+---------------------+-------------------+-------------------+------------------------+
1 |1 |957 |1 |299.98 |299.98 |
2 |2 |1073 |1 |199.99 |199.99 |
3 |2 |502 |5 |250.0 |50.0 |
4 |2 |403 |1 |129.99 |129.99 |
5 |4 |897 |2 |49.98 |24.99 |
+-------------+-------------------+---------------------+-------------------+-------------------+------------------------+
only showing top 5 rows

In [5]:
customerItemsdf = spark.read.csv("/FileStore/tables/retaildbtext/Customers.txt", sep=';', schema='customer_id int, customer_fname string, customer_lname string, customer_email string,customer_password string,customer_street string,customer_city string, customer_state string, customer_zipcode string')
customerItemsdf.show(5,False)

+-----------+--------------+--------------+--------------+-----------------+-----------------------+-------------+--------------+----------------+
customer_id|customer_fname|customer_lname|customer_email|customer_password|customer_street |customer_city|customer_state|customer_zipcode|
+-----------+--------------+--------------+--------------+-----------------+-----------------------+-------------+--------------+----------------+
1 |Richard |Hernandez |XXXXXXXXX |XXXXXXXXX |6303 Heather Plaza |Brownsville |TX |78521 |
2 |Mary |Barrett |XXXXXXXXX |XXXXXXXXX |9526 Noble Embers Ridge|Littleton |CO |80126 |
3 |Ann |Smith |XXXXXXXXX |XXXXXXXXX |3422 Blue Pioneer Bend |Caguas |PR |00725 |
4 |Mary |Jones |XXXXXXXXX |XXXXXXXXX |8324 Little Common |San Marcos |CA |92069 |
5 |Robert |Hudson |XXXXXXXXX |XXXXXXXXX |10 Crystal River Mall |Caguas |PR |00725 |
+-----------+--------------+--------------+--------------+-----------------+-----------------------+-------------+--------------+----------------+
only showing top 5 rows

In [6]:
ordersdfMap = ordersdf.select('order_id','order_date','order_customer_id')
ordersdfMap.show(5,False)

+--------+----------+-----------------+
order_id|order_date|order_customer_id|
+--------+----------+-----------------+
1 |2013-07-25|11599 |
2 |2013-07-25|256 |
3 |2013-07-25|12111 |
4 |2013-07-25|8827 |
5 |2013-07-25|11318 |
+--------+----------+-----------------+
only showing top 5 rows

In [7]:
orderItemsdfMap=orderItemsdf.select('order_item_order_id','order_item_product_id','order_item_subtotal','order_item_quantity')
orderItemsdfMap.show(5,False)

+-------------------+---------------------+-------------------+-------------------+
order_item_order_id|order_item_product_id|order_item_subtotal|order_item_quantity|
+-------------------+---------------------+-------------------+-------------------+
1 |957 |299.98 |1 |
2 |1073 |199.99 |1 |
2 |502 |250.0 |5 |
2 |403 |129.99 |1 |
4 |897 |49.98 |2 |
+-------------------+---------------------+-------------------+-------------------+
only showing top 5 rows

In [8]:
ordersdfjoin = ordersdfMap.join(orderItemsdfMap, ordersdfMap.order_id == orderItemsdfMap.order_item_order_id,'inner')
ordersdfjoin.show(5,False)


+--------+----------+-----------------+-------------------+---------------------+-------------------+-------------------+
order_id|order_date|order_customer_id|order_item_order_id|order_item_product_id|order_item_subtotal|order_item_quantity|
+--------+----------+-----------------+-------------------+---------------------+-------------------+-------------------+
1 |2013-07-25|11599 |1 |957 |299.98 |1 |
2 |2013-07-25|256 |2 |1073 |199.99 |1 |
2 |2013-07-25|256 |2 |502 |250.0 |5 |
2 |2013-07-25|256 |2 |403 |129.99 |1 |
4 |2013-07-25|8827 |4 |897 |49.98 |2 |
+--------+----------+-----------------+-------------------+---------------------+-------------------+-------------------+
only showing top 5 rows

In [9]:
ordersdfSalesmap = ordersdfjoin.select('order_date','order_customer_id','order_item_product_id','order_item_subtotal','order_item_quantity')
ordersdfSalesmap.show(10,False)

+----------+-----------------+---------------------+-------------------+-------------------+
order_date|order_customer_id|order_item_product_id|order_item_subtotal|order_item_quantity|
+----------+-----------------+---------------------+-------------------+-------------------+
2013-07-25|11599 |957 |299.98 |1 |
2013-07-25|256 |1073 |199.99 |1 |
2013-07-25|256 |502 |250.0 |5 |
2013-07-25|256 |403 |129.99 |1 |
2013-07-25|8827 |897 |49.98 |2 |
2013-07-25|8827 |365 |299.95 |5 |
2013-07-25|8827 |502 |150.0 |3 |
2013-07-25|8827 |1014 |199.92 |4 |
2013-07-25|11318 |957 |299.98 |1 |
2013-07-25|11318 |365 |299.95 |5 |
+----------+-----------------+---------------------+-------------------+-------------------+
only showing top 10 rows

In [10]:
customersdfjoin = ordersdfSalesmap.join(customerItemsdf, ordersdfSalesmap.order_customer_id == customerItemsdf.customer_id,'inner')
customersdfjoin.show(5,False)

+----------+-----------------+---------------------+-------------------+-------------------+-----------+--------------+--------------+--------------+-----------------+-------------------------+-------------+--------------+----------------+
order_date|order_customer_id|order_item_product_id|order_item_subtotal|order_item_quantity|customer_id|customer_fname|customer_lname|customer_email|customer_password|customer_street |customer_city|customer_state|customer_zipcode|
+----------+-----------------+---------------------+-------------------+-------------------+-----------+--------------+--------------+--------------+-----------------+-------------------------+-------------+--------------+----------------+
2013-07-25|11599 |957 |299.98 |1 |11599 |Mary |Malone |XXXXXXXXX |XXXXXXXXX |8708 Indian Horse Highway|Hickory |NC |28601 |
2013-07-25|256 |1073 |199.99 |1 |256 |David |Rodriguez |XXXXXXXXX |XXXXXXXXX |7605 Tawny Horse Falls |Chicago |IL |60625 |
2013-07-25|256 |502 |250.0 |5 |256 |David |Rodriguez |XXXXXXXXX |XXXXXXXXX |7605 Tawny Horse Falls |Chicago |IL |60625 |
2013-07-25|256 |403 |129.99 |1 |256 |David |Rodriguez |XXXXXXXXX |XXXXXXXXX |7605 Tawny Horse Falls |Chicago |IL |60625 |
2013-07-25|8827 |897 |49.98 |2 |8827 |Brian |Wilson |XXXXXXXXX |XXXXXXXXX |8396 High Corners |San Antonio |TX |78240 |
+----------+-----------------+---------------------+-------------------+-------------------+-----------+--------------+--------------+--------------+-----------------+-------------------------+-------------+--------------+----------------+
only showing top 5 rows

In [11]:
customersalesmap=customersdfjoin.select('order_date','order_customer_id','order_item_product_id','order_item_subtotal','order_item_quantity','customer_fname','customer_lname','customer_city','customer_state','customer_zipcode')
customersalesmap.show(10,False)

+----------+-----------------+---------------------+-------------------+-------------------+--------------+--------------+-------------+--------------+----------------+
order_date|order_customer_id|order_item_product_id|order_item_subtotal|order_item_quantity|customer_fname|customer_lname|customer_city|customer_state|customer_zipcode|
+----------+-----------------+---------------------+-------------------+-------------------+--------------+--------------+-------------+--------------+----------------+
2013-07-25|11599 |957 |299.98 |1 |Mary |Malone |Hickory |NC |28601 |
2013-07-25|256 |1073 |199.99 |1 |David |Rodriguez |Chicago |IL |60625 |
2013-07-25|256 |502 |250.0 |5 |David |Rodriguez |Chicago |IL |60625 |
2013-07-25|256 |403 |129.99 |1 |David |Rodriguez |Chicago |IL |60625 |
2013-07-25|8827 |897 |49.98 |2 |Brian |Wilson |San Antonio |TX |78240 |
2013-07-25|8827 |365 |299.95 |5 |Brian |Wilson |San Antonio |TX |78240 |
2013-07-25|8827 |502 |150.0 |3 |Brian |Wilson |San Antonio |TX |78240 |
2013-07-25|8827 |1014 |199.92 |4 |Brian |Wilson |San Antonio |TX |78240 |
2013-07-25|11318 |957 |299.98 |1 |Mary |Henry |Caguas |PR |00725 |
2013-07-25|11318 |365 |299.95 |5 |Mary |Henry |Caguas |PR |00725 |
+----------+-----------------+---------------------+-------------------+-------------------+--------------+--------------+-------------+--------------+----------------+
only showing top 10 rows

###### Problem statement-1:Get sales data for customers per day

In [13]:
customersSalesDataPerDay = customersalesmap.groupBy('order_date','customer_fname', 'customer_lname').agg(round(avg('order_item_subtotal'),2).alias("avgsales"), 
                                                                min('order_item_subtotal').alias("min_sales"),
                                                                max('order_item_subtotal').alias("max_sales"),
                                                                round(sum('order_item_subtotal'),2).alias("Totalsales"),
                                                                sum('order_item_quantity').alias("Totalquantity"),
                                                                count('order_item_subtotal').alias("Numberofsales")).sort('order_date','customer_fname', 'customer_lname','Numberofsales','Totalsales')
customersSalesDataPerDay.coalesce(1).write.csv("/FileStore/tables/customersSalesDataPerDay", compression="none", header ='true')
customersSalesDataPerDay.show(20,False)

+----------+--------------+--------------+--------+---------+---------+----------+-------------+-------------+
order_date|customer_fname|customer_lname|avgsales|min_sales|max_sales|Totalsales|Totalquantity|Numberofsales|
+----------+--------------+--------------+--------+---------+---------+----------+-------------+-------------+
2013-07-25|Albert |Jones |199.99 |199.99 |199.99 |199.99 |1 |1 |
2013-07-25|Alice |Rodriguez |93.23 |69.98 |127.96 |372.91 |11 |4 |
2013-07-25|Amy |Cummings |119.97 |119.97 |119.97 |119.97 |3 |1 |
2013-07-25|Amy |Smith |103.32 |59.99 |150.0 |309.95 |8 |3 |
2013-07-25|Brian |Wilson |174.96 |49.98 |299.95 |699.85 |14 |4 |
2013-07-25|Charles |Smith |163.94 |127.96 |199.92 |327.88 |8 |2 |
2013-07-25|Christian |Smith |124.96 |59.99 |199.95 |499.85 |15 |4 |
2013-07-25|Christina |Lewis |276.62 |129.99 |499.95 |829.86 |10 |3 |
2013-07-25|Danielle |Moran |299.98 |299.98 |299.98 |299.98 |1 |1 |
2013-07-25|David |Kerr |219.97 |129.99 |299.95 |879.86 |15 |4 |
2013-07-25|David |Rodriguez |193.33 |129.99 |250.0 |579.98 |7 |3 |
2013-07-25|Deborah |Barber |249.97 |199.99 |299.95 |499.94 |6 |2 |
2013-07-25|Diane |Davis |246.64 |39.99 |399.98 |739.92 |7 |3 |
2013-07-25|Diane |Wiggins |114.98 |99.96 |129.99 |229.95 |3 |2 |
2013-07-25|Donna |Williams |115.98 |63.98 |199.99 |463.92 |8 |4 |
2013-07-25|Doris |Smith |224.99 |99.99 |299.98 |899.94 |4 |4 |
2013-07-25|Douglas |Smith |183.31 |119.98 |299.97 |549.94 |6 |3 |
2013-07-25|Edward |Castaneda |183.32 |119.98 |299.98 |549.95 |4 |3 |
2013-07-25|Eric |Smith |209.99 |129.99 |299.98 |629.96 |3 |3 |
2013-07-25|Evelyn |Herring |199.92 |199.92 |199.92 |199.92 |4 |1 |
+----------+--------------+--------------+--------+---------+---------+----------+-------------+-------------+
only showing top 20 rows

###### Problem statement-2: Get sales data for customers per month

In [15]:
customerSalesDataPerMonth = customersalesmap.groupBy(substring('order_date',0,7).alias('ordermonth'),'customer_fname', 'customer_lname').agg(round(avg('order_item_subtotal'),2).alias("avgsales"), 
                                                                min('order_item_subtotal').alias("min_sales"),
                                                                max('order_item_subtotal').alias("max_sales"),
                                                                round(sum('order_item_subtotal'),2).alias("Totalsales"),
                                                                sum('order_item_quantity').alias("Totalquantity"),
                                                                count('order_item_subtotal').alias("Numberofsales")).sort('ordermonth','customer_fname', 'customer_lname','Numberofsales','Totalsales')
customerSalesDataPerMonth.coalesce(1).write.csv("/FileStore/tables/customerSalesDataPerMonth", compression="none", header ='true')
customerSalesDataPerMonth.show(20,False)

+----------+--------------+--------------+--------+---------+---------+----------+-------------+-------------+
ordermonth|customer_fname|customer_lname|avgsales|min_sales|max_sales|Totalsales|Totalquantity|Numberofsales|
+----------+--------------+--------------+--------+---------+---------+----------+-------------+-------------+
2013-07 |Aaron |Hanna |233.96 |119.98 |499.95 |1169.79 |18 |5 |
2013-07 |Aaron |Hudson |139.97 |119.97 |179.97 |419.92 |8 |3 |
2013-07 |Aaron |Smith |199.98 |99.96 |399.98 |799.91 |6 |4 |
2013-07 |Adam |Marquez |223.33 |120.0 |299.98 |669.98 |10 |3 |
2013-07 |Alan |Gillespie |229.99 |119.98 |399.98 |919.94 |5 |4 |
2013-07 |Alan |Weber |199.98 |99.96 |299.98 |999.9 |6 |5 |
2013-07 |Albert |Jones |199.99 |199.99 |199.99 |199.99 |1 |1 |
2013-07 |Alexander |Dominguez |143.37 |31.99 |299.98 |716.83 |11 |5 |
2013-07 |Alexander |Nguyen |239.98 |59.99 |399.98 |959.91 |8 |4 |
2013-07 |Alexander |Smith |299.98 |299.98 |299.98 |299.98 |1 |1 |
2013-07 |Alexander |Williams |249.99 |200.0 |299.98 |499.98 |5 |2 |
2013-07 |Alice |Barnes |223.3 |119.98 |399.98 |669.9 |6 |3 |
2013-07 |Alice |Rodriguez |93.23 |69.98 |127.96 |372.91 |11 |4 |
2013-07 |Alice |Smith |208.97 |29.97 |399.98 |2089.73 |22 |10 |
2013-07 |Amanda |Brown |99.96 |99.96 |99.96 |99.96 |2 |1 |
2013-07 |Amanda |Foster |243.31 |129.99 |399.98 |1459.88 |10 |6 |
2013-07 |Amanda |Gibson |257.48 |129.99 |299.98 |1029.93 |4 |4 |
2013-07 |Amanda |Hays |211.96 |119.98 |399.98 |1059.81 |13 |5 |
2013-07 |Amanda |Shaffer |249.97 |199.99 |299.95 |499.94 |6 |2 |
2013-07 |Amanda |Smith |266.66 |150.0 |399.98 |799.98 |9 |3 |
+----------+--------------+--------------+--------+---------+---------+----------+-------------+-------------+
only showing top 20 rows

###### Problem statement-3: Get sales data  for customers per year

In [17]:
customerSalesDataPerYear = customersalesmap.groupBy(substring('order_date',0,4).alias('orderyear'),'customer_fname', 'customer_lname').agg(round(avg('order_item_subtotal'),2).alias("avgsales"), 
                                                                min('order_item_subtotal').alias("min_sales"),
                                                                max('order_item_subtotal').alias("max_sales"),
                                                                round(sum('order_item_subtotal'),2).alias("Totalsales"),
                                                                sum('order_item_quantity').alias("Totalquantity"),
                                                                count('order_item_subtotal').alias("Numberofsales")).sort('orderyear','customer_fname', 'customer_lname','Numberofsales','Totalsales')
customerSalesDataPerYear.coalesce(1).write.csv("/FileStore/tables/customerSalesDataPerYear", compression="none", header ='true')
customerSalesDataPerYear.show(20,False)

+---------+--------------+--------------+--------+---------+---------+----------+-------------+-------------+
orderyear|customer_fname|customer_lname|avgsales|min_sales|max_sales|Totalsales|Totalquantity|Numberofsales|
+---------+--------------+--------------+--------+---------+---------+----------+-------------+-------------+
2013 |Aaron |Berger |214.41 |100.0 |399.98 |1929.72 |20 |9 |
2013 |Aaron |Boyle |113.33 |39.99 |250.0 |679.98 |14 |6 |
2013 |Aaron |Bush |279.97 |119.97 |399.98 |1119.89 |10 |4 |
2013 |Aaron |Carr |179.06 |39.99 |299.98 |1969.71 |32 |11 |
2013 |Aaron |Cline |256.41 |44.99 |399.98 |1794.89 |11 |7 |
2013 |Aaron |Davis |197.98 |50.0 |399.98 |989.92 |8 |5 |
2013 |Aaron |Hanna |171.08 |29.99 |499.95 |3079.47 |41 |18 |
2013 |Aaron |Hudson |253.31 |50.0 |499.95 |4559.49 |56 |18 |
2013 |Aaron |Kennedy |151.23 |74.97 |299.95 |604.91 |11 |4 |
2013 |Aaron |Mann |99.99 |99.99 |99.99 |99.99 |1 |1 |
2013 |Aaron |Mcmahon |233.29 |50.0 |399.98 |699.88 |7 |3 |
2013 |Aaron |Norman |195.98 |49.98 |399.98 |979.92 |6 |5 |
2013 |Aaron |Riddle |231.35 |50.0 |399.98 |2544.81 |19 |11 |
2013 |Aaron |Salazar |145.18 |99.96 |239.96 |725.88 |12 |5 |
2013 |Aaron |Sharp |219.98 |199.98 |299.98 |1099.92 |7 |5 |
2013 |Aaron |Smith |189.61 |49.98 |399.98 |14410.4 |141 |76 |
2013 |Aaron |Valdez |399.98 |399.98 |399.98 |1199.94 |3 |3 |
2013 |Aaron |Villa |174.98 |50.0 |299.98 |1399.8 |20 |8 |
2013 |Aaron |Williamson |139.99 |89.98 |199.99 |419.97 |5 |3 |
2013 |Adam |Blair |304.97 |24.99 |499.95 |3354.68 |21 |11 |
+---------+--------------+--------------+--------+---------+---------+----------+-------------+-------------+
only showing top 20 rows

###### Problem statement-4:Get monthly sales data from customers visited more than 100 times

In [19]:
customervisitDataPerMonth = customersalesmap.groupBy(substring('order_date',0,7).alias("ordermonth"),'customer_fname', 'customer_lname').agg(round(avg('order_item_subtotal'),2).alias("avgsales"), 
                                                                count('order_customer_id').alias("customervisit_permonth"), 
                                                                min('order_item_subtotal').alias("min_sales"),
                                                                max('order_item_subtotal').alias("max_sales"),
                                                                round(sum('order_item_subtotal'),2).alias("Totalsales"),
                                                                sum('order_item_quantity').alias("Totalquantity_perstatus"),
                                                                count('order_item_subtotal').alias("Numberofsales_perstatus")).\
                                                                filter(count('order_customer_id')>100).\
                                                                sort('ordermonth','customer_fname','customer_lname','customervisit_permonth')
customervisitDataPerMonth.coalesce(1).write.csv("/FileStore/tables/customervisitDataPerMonth", compression="none", header ='true')
customervisitDataPerMonth.show(20,False)

+----------+--------------+--------------+--------+----------------------+---------+---------+----------+-----------------------+-----------------------+
ordermonth|customer_fname|customer_lname|avgsales|customervisit_permonth|min_sales|max_sales|Totalsales|Totalquantity_perstatus|Numberofsales_perstatus|
+----------+--------------+--------------+--------+----------------------+---------+---------+----------+-----------------------+-----------------------+
2013-07 |Mary |Smith |199.07 |545 |17.99 |499.95 |108494.36 |1241 |545 |
2013-08 |Mary |Smith |198.8 |2048 |9.99 |500.0 |407147.15 |4575 |2048 |
2013-09 |Mary |Smith |199.43 |2063 |14.99 |999.99 |411423.71 |4524 |2063 |
2013-10 |Mary |Smith |199.18 |1833 |9.99 |599.99 |365103.5 |4095 |1833 |
2013-11 |Mary |Smith |198.64 |2224 |14.99 |499.95 |441765.2 |4816 |2224 |
2013-12 |James |Smith |206.41 |101 |25.0 |399.98 |20847.77 |215 |101 |
2013-12 |Mary |Smith |197.38 |1929 |9.99 |599.99 |380748.94 |4176 |1929 |
2014-01 |Mary |Smith |198.79 |2040 |9.99 |999.99 |405525.21 |4371 |2040 |
2014-02 |Mary |Smith |198.92 |1956 |9.99 |500.0 |389095.64 |4185 |1956 |
2014-03 |Mary |Smith |200.52 |1969 |15.99 |599.99 |394832.59 |4281 |1969 |
2014-03 |Robert |Smith |196.06 |108 |19.99 |499.95 |21175.0 |197 |108 |
2014-04 |Mary |Smith |201.45 |1948 |14.99 |499.95 |392422.05 |4269 |1948 |
2014-05 |Mary |Smith |202.17 |1974 |15.99 |499.95 |399081.01 |4334 |1974 |
2014-06 |Mary |Smith |203.93 |1790 |9.99 |500.0 |365037.06 |3976 |1790 |
2014-07 |Mary |Smith |198.7 |1565 |9.99 |499.95 |310969.49 |3347 |1565 |
+----------+--------------+--------------+--------+----------------------+---------+---------+----------+-----------------------+-----------------------+

###### Problem statement-5:Get monthly sales data from Top10 revenue generating customers

In [21]:
customeDataTop10revenue = customersalesmap.groupBy('customer_fname', 'customer_lname').agg( round(sum('order_item_subtotal'),2).alias("customer_revenue")).sort(col('customer_revenue').desc()).limit(10)
customeDataTop10revenue.coalesce(1).write.csv("/FileStore/tables/customeDataTop10revenue", compression="none", header ='true')
customeDataTop10revenue.show(20,False)

+--------------+--------------+----------------+
customer_fname|customer_lname|customer_revenue|
+--------------+--------------+----------------+
Mary |Smith |4771645.88 |
Robert |Smith |152230.96 |
James |Smith |147924.43 |
David |Smith |145776.8 |
John |Smith |129882.58 |
William |Smith |108523.28 |
Michael |Smith |92843.03 |
Elizabeth |Smith |90516.02 |
Mary |Jones |88860.48 |
Christopher |Smith |85271.05 |
+--------------+--------------+----------------+

###### Problem statement-6:Get daily sales data from fourth quarter of 2013 and second quarter of 2014 where and Numberofsales>=110 in state of CA

In [23]:
SalesDataPerMonth=customersalesmap.where((to_date(customersalesmap.order_date) >=lit("2013-10-01")) & 
                                                 (to_date(customersalesmap.order_date) <=lit("2013-12-31")) |
                                                 (to_date(customersalesmap.order_date) >=lit("2014-04-01")) & 
                                                 (to_date(customersalesmap.order_date) <=lit("2014-06-30")))
SalesDatacanceledordersquarter = SalesDataPerMonth.groupBy(substring('order_date',0,10).alias("orderdate"),'customer_state').agg(round(avg('order_item_subtotal'),2).alias("avgsales"), 
                                                                min('order_item_subtotal').alias("min_sales"),
                                                                max('order_item_subtotal').alias("max_sales"),
                                                                round(sum('order_item_subtotal'),2).alias("Totalsales"),
                                                                sum('order_item_quantity').alias("Totalquantity_perstatus"),
                                                                count('order_item_subtotal').alias("Numberofsales")).\
                                                                filter((count('order_item_subtotal')>=110) & (SalesDataPerMonth.customer_state == 'CA')).\
                                                                sort('orderdate')
SalesDatacanceledordersquarter.coalesce(1).write.csv("/FileStore/tables/SalesDatacanceledordersquarter", compression="none", header ='true')
SalesDatacanceledordersquarter.show(30,False)

+----------+--------------+--------+---------+---------+----------+-----------------------+-------------+
orderdate |customer_state|avgsales|min_sales|max_sales|Totalsales|Totalquantity_perstatus|Numberofsales|
+----------+--------------+--------+---------+---------+----------+-----------------------+-------------+
2013-10-06|CA |188.68 |24.99 |399.98 |26038.14 |286 |138 |
2013-11-03|CA |206.17 |24.99 |499.95 |35462.01 |378 |172 |
2013-11-05|CA |204.6 |49.98 |499.95 |25370.88 |304 |124 |
2013-11-07|CA |207.98 |25.0 |499.95 |23501.35 |230 |113 |
2013-11-11|CA |189.18 |24.99 |399.99 |25539.55 |303 |135 |
2013-11-14|CA |203.49 |39.99 |499.95 |22587.48 |248 |111 |
2013-11-23|CA |205.73 |25.0 |499.95 |28184.84 |321 |137 |
2013-11-24|CA |193.26 |19.99 |499.95 |26089.8 |271 |135 |
2013-12-18|CA |198.05 |17.99 |500.0 |29707.34 |336 |150 |
2013-12-20|CA |192.64 |15.99 |499.95 |21190.68 |239 |110 |
2013-12-21|CA |194.51 |17.99 |399.98 |21396.22 |239 |110 |
2013-12-26|CA |199.33 |25.0 |999.99 |30697.02 |323 |154 |
2014-04-21|CA |193.06 |35.98 |499.95 |21622.29 |248 |112 |
2014-04-23|CA |193.58 |14.99 |499.95 |24004.27 |262 |124 |
2014-05-02|CA |192.1 |34.99 |499.95 |21322.58 |242 |111 |
2014-05-12|CA |218.66 |34.99 |499.95 |24708.23 |245 |113 |
2014-06-02|CA |208.16 |19.99 |499.95 |23938.49 |247 |115 |
2014-06-08|CA |190.78 |15.99 |399.98 |20986.06 |251 |110 |
2014-06-09|CA |175.94 |21.99 |499.95 |21992.26 |295 |125 |
+----------+--------------+--------+---------+---------+----------+-----------------------+-------------+

###### Problem statement-7:Get Top10 revenue generating cities

In [25]:
customeDataTop10revcities = customersalesmap.groupBy('customer_city').agg( round(sum('order_item_subtotal'),2).alias("city_revenue")).sort(col('city_revenue').desc()).limit(10)
customeDataTop10revcities.coalesce(1).write.csv("/FileStore/tables/customeDataTop10revcities", compression="none", header ='true')
customeDataTop10revcities.show(20,False)

+-------------+-------------+
customer_city|city_revenue |
+-------------+-------------+
Caguas |1.269771832E7|
Chicago |746869.58 |
Los Angeles |645083.24 |
Brooklyn |643507.93 |
New York |338937.58 |
Philadelphia |297670.29 |
Bronx |288041.72 |
San Diego |279678.07 |
Miami |253232.68 |
Houston |248282.44 |
+-------------+-------------+

###### Problem statement-8:Get  Top10 revenue generating zipcodes and its corresponding cities and states

In [27]:
customeDataTop10revcities = customersalesmap.groupBy('customer_zipcode','customer_city','customer_state'). \
                            agg( round(sum('order_item_subtotal'),2).alias("zipcode_revenue")).sort(col('zipcode_revenue').desc()).limit(10)
customeDataTop10revcities.coalesce(1).write.csv("/FileStore/tables/customeDataTop10revcities", compression="none", header ='true')
customeDataTop10revcities.show(20,False)

+----------------+---------------+--------------+---------------+
customer_zipcode|customer_city |customer_state|zipcode_revenue|
+----------------+---------------+--------------+---------------+
00725 |Caguas |PR |1.269771832E7 |
23455 |Virginia Beach |VA |66357.08 |
33324 |Fort Lauderdale|FL |57030.12 |
79109 |Amarillo |TX |55147.6 |
92024 |Encinitas |CA |50634.74 |
33624 |Tampa |FL |50504.17 |
78586 |San Benito |TX |50280.56 |
11226 |Brooklyn |NY |49229.64 |
80012 |Aurora |CO |49203.59 |
92115 |San Diego |CA |48850.26 |
+----------------+---------------+--------------+---------------+

###### Problem statement-9:Get revenue of  each zipcodes of state COLORADO and its corresponding cities

In [29]:
zipcoderevenueDataCOstate = customersalesmap.groupBy('customer_zipcode','customer_city','customer_state'). \
                            agg(round(sum('order_item_subtotal'),2).alias("zipcode_revenue")).filter(customersalesmap.customer_state=='CO').sort(col('customer_zipcode'))
zipcoderevenueDataCOstate.coalesce(1).write.csv("/FileStore/tables/zipcoderevenueDataCOstate", compression="none", header ='true')
zipcoderevenueDataCOstate.show(20,False)

+----------------+----------------+--------------+---------------+
customer_zipcode|customer_city |customer_state|zipcode_revenue|
+----------------+----------------+--------------+---------------+
80010 |Aurora |CO |21922.41 |
80011 |Aurora |CO |22319.24 |
80012 |Aurora |CO |49203.59 |
80013 |Aurora |CO |24281.22 |
80015 |Aurora |CO |16424.21 |
80020 |Broomfield |CO |12344.44 |
80110 |Englewood |CO |13481.41 |
80126 |Littleton |CO |27356.53 |
80219 |Denver |CO |29316.54 |
80231 |Denver |CO |16529.28 |
80501 |Longmont |CO |37619.57 |
80631 |Greeley |CO |40771.26 |
80906 |Colorado Springs|CO |22490.68 |
80918 |Colorado Springs|CO |24250.22 |
+----------------+----------------+--------------+---------------+

###### Problem statement-10:Get total revenue of each state

In [31]:
SalesDatarevenueperstate = customersalesmap.groupBy('customer_state'). \
                            agg(round(sum('order_item_subtotal'),2).alias("state_revenue")).sort(col('customer_state'))
SalesDatarevenueperstate.coalesce(1).write.csv("/FileStore/tables/SalesDatarevenueperstate", compression="none", header ='true')
SalesDatarevenueperstate.show(55,False)

+--------------+-------------+
customer_state|state_revenue|
+--------------+-------------+
AL |6401.4 |
AR |29241.61 |
AZ |566459.3 |
CA |5542723.11 |
CO |358310.61 |
CT |211264.24 |
DC |108841.48 |
DE |48873.25 |
FL |1048609.79 |
GA |467765.19 |
HI |238208.11 |
IA |12761.74 |
ID |31791.2 |
IL |1457225.86 |
IN |110015.14 |
KS |85865.07 |
KY |94894.75 |
LA |182954.74 |
MA |306025.74 |
MD |456100.43 |
MI |730078.98 |
MN |133183.97 |
MO |260417.29 |
MT |16189.98 |
NC |378877.65 |
ND |41221.97 |
NJ |606551.0 |
NM |181366.65 |
NV |276364.98 |
NY |2152706.78 |
OH |773804.12 |
OK |43493.32 |
OR |315239.52 |
PA |724375.94 |
PR |1.320886795E7|
RI |47439.35 |
SC |129411.79 |
TN |297614.42 |
TX |1731407.52 |
UT |184356.83 |
VA |344824.36 |
WA |172982.45 |
WI |164058.32 |
WV |43452.67 |
+--------------+-------------+

###### Problem statement-11:Get the list of inactive customers

In [33]:
listofInactivecustomers = customerItemsdf.join(ordersdf,customerItemsdf.customer_id == ordersdf.order_customer_id,"LeftAnti")
listofInactivecustomers.coalesce(1).write.csv("/FileStore/tables/listofInactivecustomers", compression="none", header ='true')
listofInactivecustomers.show(20,False)

+-----------+--------------+--------------+--------------+-----------------+----------------------------+-------------------+--------------+----------------+
customer_id|customer_fname|customer_lname|customer_email|customer_password|customer_street |customer_city |customer_state|customer_zipcode|
+-----------+--------------+--------------+--------------+-----------------+----------------------------+-------------------+--------------+----------------+
219 |Mary |Harrell |XXXXXXXXX |XXXXXXXXX |9016 Foggy Robin Expressway |Denver |CO |80219 |
339 |Mary |Greene |XXXXXXXXX |XXXXXXXXX |4271 Hazy Close |Long Beach |CA |90805 |
469 |Randy |Smith |XXXXXXXXX |XXXXXXXXX |252 Golden Goose Loop |South San Francisco|CA |94080 |
1187 |Dorothy |Vazquez |XXXXXXXXX |XXXXXXXXX |363 Green Goose Run |Danbury |CT |06810 |
1481 |Grace |Smith |XXXXXXXXX |XXXXXXXXX |2171 Clear Lake Isle |Caguas |PR |00725 |
1808 |Albert |Ellison |XXXXXXXXX |XXXXXXXXX |9795 Heather Wynd |Billings |MT |59102 |
2073 |Donna |Stephens |XXXXXXXXX |XXXXXXXXX |9792 Cozy Corners |Sunnyvale |CA |94087 |
2096 |Jose |Tanner |XXXXXXXXX |XXXXXXXXX |8976 Old Hickory Landing |Bronx |NY |10467 |
2450 |James |Smith |XXXXXXXXX |XXXXXXXXX |4063 Little Creek Court |Newark |DE |19702 |
4555 |Mary |Smith |XXXXXXXXX |XXXXXXXXX |5455 Red Lagoon Maze |Caguas |PR |00725 |
4927 |Carolyn |Green |XXXXXXXXX |XXXXXXXXX |7550 Sleepy View Court |Caguas |PR |00725 |
6072 |Stephen |Smith |XXXXXXXXX |XXXXXXXXX |849 Noble Apple Private |Malden |MA |02148 |
6613 |Ashley |Smith |XXXXXXXXX |XXXXXXXXX |9847 Dusty Horse Corner |Caguas |PR |00725 |
7011 |Kevin |Smith |XXXXXXXXX |XXXXXXXXX |1915 Thunder Hickory Freeway|Wyandotte |MI |48192 |
7552 |Carl |Smith |XXXXXXXXX |XXXXXXXXX |9966 Cinder Loop |Howell |MI |48843 |
8243 |Gary |Walker |XXXXXXXXX |XXXXXXXXX |2447 Stony Barn Street |New York |NY |10128 |
8343 |Mary |Bolton |XXXXXXXXX |XXXXXXXXX |7302 Sunny Valley |Caguas |PR |00725 |
8575 |Mary |Mueller |XXXXXXXXX |XXXXXXXXX |9714 Emerald Bear Lookout |Caguas |PR |00725 |
8778 |Mary |Smith |XXXXXXXXX |XXXXXXXXX |4015 Tawny Rise Crescent |Caguas |PR |00725 |
8882 |Kenneth |Smith |XXXXXXXXX |XXXXXXXXX |6754 Iron Leaf Line |Hickory |NC |28601 |
+-----------+--------------+--------------+--------------+-----------------+----------------------------+-------------------+--------------+----------------+
only showing top 20 rows